# Week 1: Building Your RAG Evaluation Foundation

Evaluating RAG systems is challenging, especially when you're just starting out. This notebook introduces a practical approach to assessment using synthetic data generation - a crucial first step before diving into more complex metrics.

## Why This Matters

Traditional RAG evaluation focuses on generated content quality, but this approach has significant drawbacks:

| Aspect | Content Generation | Retrieval Metrics |
|--------|-------------------|-------------------|
| Speed | 1-10s per test | 10-800ms per test |
| Cost | $100s per run | Negligible |
| Objectivity | Subjective | Quantitative |
| Iteration Speed | Hours | Minutes |
| Scale | Limited | Automated |

Instead, we'll focus on retrieval metrics that are:
- Fast to compute (milliseconds vs seconds)
- Objective and reproducible
- Easy to automate
- Cost-effective at scale

## What You'll Learn

Through this hands-on tutorial, you'll learn to build a comprehensive evaluation framework:

1. **Synthetic Data Generation**
   - Create diverse, realistic test questions
   - Generate comprehensive datasets without real user data
   - Learn techniques for systematic question generation

2. **Maintain Query Diversity**
   - Identify different types of questions to test
   - Cover various query patterns and edge cases
   - Build representative test scenarios

3. **Evaluation Setup**
   - Establish measurement foundations
   - Set up automated testing pipelines
   - Create reproducible evaluation workflows

By the end of this notebook, you'll have a good understanding of what retrieval metrics are, how we can generate synthetic questions to benchmark our retrieval system and how we can use these questions to evaluate our retrieval system.


## Open Source Models

> Make sure that you've deployed your vLLM server to Modal before continuing with this notebook. You can do so by running the following commands from the `week1` folder in your terminal
> ```
> modal run modal_vllm.py
> modal deploy modal_vllm.py
> ```
> 
> The initial download of the model should be relatively fast. Expect startup to take ~3-5 minutes

For this notebook, we'll use `Qwen-2.5-7B-Instruct`, a powerful open-source model that's cost-effective for generating synthetic evaluation data. While commercial APIs like OpenAI provide excellent quality, using open-source models offers several advantages:

Cost control: Generate thousands of test questions without expensive API fees
Data privacy: Keep all your data within your infrastructure
Customizability: Fine-tune models for your specific domain if needed



In [ ]:
from openai import OpenAI
from rich import print as rprint

# Modal will give you the unique URL, add a /v1 behind it so that we're using the correct endpoint on your server
BASE_URL = "https://ivanleomk--vllm-server-serve-dev.modal.run/v1"
API_KEY = "super-secret-key"

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
)

resp = client.chat.completions.create(
    model="Qwen/Qwen2.5-7B-Instruct",
    messages=[{"role": "user", "content": "Ivan is 28 years old?"}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "User",
                "description": "Correctly extracted `User` with all the required parameters with correct types",
                "parameters": {
                    "properties": {
                        "name": {"title": "Name", "type": "string"},
                        "age": {"title": "Age", "type": "string"},
                    },
                    "required": ["age", "name"],
                    "type": "object",
                },
            },
        }
    ],
    tool_choice={
        "type": "function",
        "function": {
            "name": "User",
        },
    },
)

rprint(resp.choices[0].message.tool_calls[0].function.arguments)

<tool_call>
{"name": "User", "arguments": {"name": "Ivan", "age": "28"}}
</tool_call>

We can see that the vLLM server returns to us the function arguments in a `<tool_call>` XML tag. Therefore, in order to parse it into a Pydantic object, we can override the BaseModel class so that we can remove these XML tags before validating the JSON object.

In [7]:
from pydantic import BaseModel
from typing import Any, Type
import json


class XMLBaseModel(BaseModel):
    @classmethod
    def model_validate_json(
        cls: Type[BaseModel], json_data: str, **kwargs: Any
    ) -> BaseModel:
        # Custom logic before validation
        json_data = json_data.replace("<tool_call>\n", "").replace("\n</tool_call>", "")
        tool_call_payload = json.dumps(json.loads(json_data)["arguments"])

        # Call the original model_validate_json for standard validation
        instance = super().model_validate_json(tool_call_payload, **kwargs)
        return instance


class User(XMLBaseModel):
    name: str
    age: int


user = User.model_validate_json(
    resp.choices[0].message.tool_calls[0].function.arguments
)
rprint(user)

User(name='Ivan', age=28)

Now that we've figured out how to parse the returned values from our vLLM server, let's see how we can use `instructor` with our new OpenAI Client in order to get a validated User object.

In [12]:
import instructor
from openai import AsyncOpenAI

client = instructor.from_openai(
    AsyncOpenAI(
        api_key=API_KEY,
        base_url=BASE_URL
    )
)

resp = await client.chat.completions.create(
    model="Qwen/Qwen2.5-7B-Instruct",
    messages=[{"role": "user", "content": "Ivan is 28 years old?"}],
    response_model=User,
)
print(resp)

name='Ivan' age=28



## Evaluating Retrieval

Before looking at our case study, let's first understand some of the metrics that we'll be using to evaluate our retrieval system. These metrics will form the basis of our evaluation framework throughout this course.

### Key Retrieval Metrics

**Precision** measures how many of our retrieved items are actually relevant:

$$ \text{Precision} = \frac{\text{Number of Relevant Items Retrieved}}{\text{Total Number of Retrieved Items}} $$ 

For example, if your system retrieves 10 documents but only 5 are relevant, that's 50% precision. Low precision indicates your system is wasting resources processing irrelevant content.

**Recall** measures how many of the total relevant items we managed to find:

$$ \text{Recall} = \frac{\text{Number of Relevant Items Retrieved}}{\text{Total Number of Relevant Items}} $$ 

If there are 20 relevant documents in your database but you only retrieve 10 of them, that's 50% recall. Low recall suggests you're missing important information.

In practice, we often measure these metrics at specific cutoff points (like top-5 or top-10 results), denoted as Precision@K or Recall@K. This reflects real-world usage where users typically only look at the first few results.

Consider an example where we're trying to build a Text-2-SQL application. This is an application where we take in a user query and output a SQL query which can be used to retrieve the relevant information as seen below.

```
Text : Hey could you help me find the top 5 most popular items in our store?
Query: SELECT item_name, COUNT(*) as popularity FROM items GROUP BY item_name ORDER BY popularity DESC LIMIT 5
```

This feels like a generation task but ultimately can be greatly improved by including few-shot examples. This is very similar to how we might look for relevant text chunks in a classic Question Answer RAG application.

However,by framing this as a retrieval task, we can start by looking at precision and recall of our retrieval system before we even start looking at the generated SQL queries. This has two main benefits.

1. When we do evaluate the generated SQL queries, we can identify edge cases early on and add them to our list of snippets. We can then verify that these few shot examples are retrieved when we encounter these questions to help generate better SQL snippets.
2. Different companies have unique business logic or calculation methods. Being able to retrieve the relevant snippets when these specific measurements are required is crucial. 


## Case Study : Bird-Bench

For this case-study, we'll be using the Bird-Bench dataset. This is a huge Text-2-SQL dataset which contains a collection of text questions to a corresponding sql query. 

We'll be using the dev split of this dataset for this case study that provides ~1500+ sql snippets that involves ~95 different tables that we can use. 

We've cleaned the dataset ahead of time and uploaded it to `567-labs/bird-rag`. Each example in our dataset contains three things

- `id` : This is a unique identifier for each query
- `query` : This is a sample SQL query 
- `difficulty` : This is a label that indicates how difficult the query is to generate. It can be either `simple`, `moderate` or `challenging`. 

For this case study, we'll only be using the `challenging` queries so that we can generate more difficult questions. This allows us to test our retrieval system under more demanding conditions, ensuring that it performs well even with complex queries

With that in mind, let's take a look at our dataset

In [1]:
import datasets
from rich import print

dataset = datasets.load_dataset("567-labs/bird-rag")["train"]

print(dataset[0])
for item in dataset:
    if item["difficulty"] == "challenging":
        print(item["query"])
        break

/Users/ivanleo/Documents/coding/systematically-improving-rag/cohort_2/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


{
    'id': '0',
    'query': "SELECT `Free Meal Count (K-12)` / `Enrollment (K-12)` FROM frpm WHERE `County Name` = 'Alameda' ORDER
BY (CAST(`Free Meal Count (K-12)` AS REAL) / `Enrollment (K-12)`) DESC LIMIT 1",
    'difficulty': 'simple'
}

SELECT T2.School, T2.DOC FROM frpm AS T1 INNER JOIN schools AS T2 ON T1.CDSCode = T2.CDSCode WHERE T2.FundingType =
'Locally funded' AND (T1.`Enrollment (K-12)` - T1.`Enrollment (Ages 5-17)`) > (SELECT AVG(T3.`Enrollment (K-12)` - 
T3.`Enrollment (Ages 5-17)`) FROM frpm AS T3 INNER JOIN schools AS T4 ON T3.CDSCode = T4.CDSCode WHERE 
T4.FundingType = 'Locally funded')

Let's analyze a sample SQL query to understand what kind of synthetic questions we can generate:

> SELECT `Free Meal Count (K-12)` / `Enrollment (K-12)` FROM frpm WHERE `County Name` = 'Alameda' ORDER BY (CAST(`Free Meal Count (K-12)` AS REAL) / `Enrollment (K-12)`) DESC LIMIT 1

This query:
1. Calculates the percentage of students receiving free meals
2. Filters to Alameda County schools only
3. Returns the school with the highest percentage

Some relevant natural language questions could be:

- "What school in Alameda County has the highest proportion of students on free meal programs?"
- "Which school has the highest free meal participation rate in Alameda County?"

By generating a dataset of similar questions, we can evaluate how well our retrieval system matches user queries to the appropriate SQL snippets.

## Generating Synthetic Questions

Now let's start generating our synthetic questions. We're going to begin by defining some Pydantic models that represent the format of the data that we're working with.

We're doing so because of the following reasons

1. It helps us to be explicit about the data we're working with 
2. We can use these models with the `instructor` library to obtain structured outputs from our LLM calls


In [15]:
# This represents how we're representing our data from the dataset
class Chunk(BaseModel):
    chunk_id: str
    text: str


# This is the synthetic question that we want our model to generate
class Question(XMLBaseModel):
    chain_of_thought: str
    question: str


# This represents a single question-chunk pair that we'll be using for our evaluation later on
class ChunkEval(BaseModel):
    chunk_id: str
    question: str
    chunk: str

We're using `instructor` because it handles prompt templating with `jinja` for us and provides validated structured outputs. 

All we need to do is to define a Pydantic model that represents a desired output and the library will handle the rest. 

Remember that we want to generate a question that should either be answerable by the data returned by the SQL snippet directly or with some small tweaks.



In [16]:
import openai
import instructor
from asyncio import Semaphore
from tqdm.asyncio import tqdm_asyncio as asyncio
from tenacity import retry, stop_after_attempt, wait_fixed
from rich import print

client = openai.OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
)
client = instructor.from_openai(client)

sql_snippet = """\
SELECT `Free Meal Count (K-12)` / `Enrollment (K-12)` 
FROM frpm 
WHERE `County Name` = 'Alameda' 
ORDER BY (CAST(`Free Meal Count (K-12)` AS REAL) / `Enrollment (K-12)`) DESC 
LIMIT 1"""

resp = client.chat.completions.create(
    model="Qwen/Qwen2.5-7B-Instruct",
    messages=[
        {
            "role": "user",
            "content": """
        Generate a hypothetical question that can be answered using the following SQL snippet. 

        SQL Snippet:
        {{ snippet }}

        Rules
        - If there are specific values in the snippet, do not use them directly in the question if possible. 
        - The question should be at most 2 sentences long
        - if necessary, consider making the question more challenging using the following constraint - If there's a time period mentioned in the snippet, modify it slightly (Eg. if the snippet is looking at the entire year, change it to 6 months or 1.5 years)
        - The question must be answerable using the SQL snippet or at most with a small tweak
        """,
        }
    ],
    response_model=Question,
    context={
        "snippet": sql_snippet
    },  # This is the context that we're passing to the model
)

print(resp.question)

Which county had the highest rate of free meals per student in the second half of the year?

> `Which county had the highest rate of free meals per student in the second half of the year?`

This is a question which the SQL snippet would be highly relevant for. In order to answer this query, we would just need to add a groupBy instead of using a filter on the `county` value as well as include a new time based filter

### The Diversity Problem

We cannot use the same prompt and expect a diverse set of questions. Therefore we need to introduce slight variations in the prompt to generate questions that are different in wording, intent and content. This is crucial in identifying blindspots in our retrieval system. 

In the example below, we're using the same prompt but introducing randomly chosen constraints at each point. This forces the model to write and generate different questions each time, allowing us to collect a more diverse set of questions. The key here is to really introduce different sources of variation when doing these generations.

### Scaling Up Our Questions

With those points in mind, let's scale our question generation up.

We'll do so by generating a question for each SQL snippet marked as challenging. Since this will be a large number of requests, we're going to be doing so asynchronously with the `asyncio` library.

Additionally, to make sure we stay within our rate limits , we'll be using a semaphore to limit the number of concurrent requests.

We're also making sure that we have a good diversity of questions by randomly selecting a constraint from a set of constraints to make the question more challenging.

In [7]:
import random
from tqdm.asyncio import tqdm_asyncio
import asyncio

# Define Instructor Client
client = instructor.from_openai(
    openai.AsyncOpenAI(
        api_key=API_KEY,
        base_url=BASE_URL,
    )
)

# Define some constraints to make the question more challenging
constraints = [
    "If there's a time period mentioned in the snippet, modify it slightly (Eg. if the snippet is looking at the entire year, change it to 6 months or 1.5 years)",
    "Add in some irrelevant context (Eg. Add information about the weather, a random event or a backstory that isn't mentioned in the snippet)",
    "Changing the value of the filter (Eg. if the snippet is looking at the results in Canada, change the question to ask about another country or city instead)",
]


@retry(stop=stop_after_attempt(3), wait=wait_fixed(10))
async def generate_questions(chunk: Chunk, sem: Semaphore) -> ChunkEval:
    async with sem:
        coro = client.chat.completions.create(
            model="Qwen/Qwen2.5-7B-Instruct",
            messages=[
                {
                    "role": "user",
                    "content": """
                Generate a hypothetical question that can be answered using the following SQL snippet. 

                SQL Snippet:
                {{ snippet }}

                Rules
                - If there are specific values in the snippet, do not use them directly in the question if possible. 
                - The question should be at most 2 sentences long
                - if necessary, consider making the question more challenging using the following constraint of {{ constraint }}
                - The question must be answerable using the SQL snippet or at most with a small tweak
                """,
                }
            ],
            response_model=Question,
            context={"snippet": chunk.text, "constraint": random.choice(constraints)},
        )
        resp = await asyncio.wait_for(coro, timeout=30)

        return ChunkEval(
            chunk_id=chunk.chunk_id,
            question=resp.question,
            chunk=chunk.text,
        )


sem = Semaphore(10)
dataset = [
    item
    for item in datasets.load_dataset("567-labs/bird-rag")["train"]
    if item["difficulty"] == "challenging"
]
dataset = [Chunk(chunk_id=item["id"], text=item["query"]) for item in dataset]

coros = []

num_samples = 2
for chunk in dataset:
    for _ in range(num_samples):
        coros.append(generate_questions(chunk, sem))

questions: list[ChunkEval] = await tqdm_asyncio.gather(*coros)

100%|██████████| 290/290 [01:11<00:00,  4.07it/s]


Now that we've generated our questions, let's take a look at what they look like.

In [8]:
from rich import print


for i in range(2):
    print(
        f"""
    Question: {questions[i].question}

    SQL Snippet: {questions[i].chunk}
    """
    )

Question: In a region with diverse weather conditions, which schools, that are locally funded and have more 
students in K-12 than in Ages 5-17 by a significant margin compared to the average, located in sunny areas, would 
you consider for a summer program?

    SQL Snippet: SELECT T2.School, T2.DOC FROM frpm AS T1 INNER JOIN schools AS T2 ON T1.CDSCode = T2.CDSCode WHERE
T2.FundingType = 'Locally funded' AND (T1.`Enrollment (K-12)` - T1.`Enrollment (Ages 5-17)`) > (SELECT 
AVG(T3.`Enrollment (K-12)` - T3.`Enrollment (Ages 5-17)`) FROM frpm AS T3 INNER JOIN schools AS T4 ON T3.CDSCode = 
T4.CDSCode WHERE T4.FundingType = 'Locally funded')

Question: Which schools, among those locally funded, have a higher difference between their K-12 enrollment and
Ages 5-17 enrollment compared to the average difference for all locally funded schools?

    SQL Snippet: SELECT T2.School, T2.DOC FROM frpm AS T1 INNER JOIN schools AS T2 ON T1.CDSCode = T2.CDSCode WHERE
T2.FundingType = 'Locally funded' AND (T1.`Enrollment (K-12)` - T1.`Enrollment (Ages 5-17)`) > (SELECT 
AVG(T3.`Enrollment (K-12)` - T3.`Enrollment (Ages 5-17)`) FROM frpm AS T3 INNER JOIN schools AS T4 ON T3.CDSCode = 
T4.CDSCode WHERE T4.FundingType = 'Locally funded')

Looking at both questions, they target the same data - locally funded schools with above-average enrollment differences. However, the first question adds contextual elements about weather conditions and summer programs that aren't in the SQL query, while the second question directly matches the query's purpose.

Creating such variations with different contexts, time frames, or locations helps test how well our retrieval system handles diverse phrasings of essentially the same core question, which is crucial for uncovering potential blindspots.

## Saving our Questions

Pydantic AI makes it easy for us to load and save datasets. We'll be taking advantage of that to save our data to a `.yaml` file

In [9]:
from pydantic_evals import Case, Dataset

cases = [
    Case(
        name=f"question_{index}",
        inputs=question.question,
        expected_output=[question.chunk_id],
        metadata={"chunk_id": question.chunk},
    )
    for index, question in enumerate(questions)
]

dataset = Dataset(cases=cases)
dataset.to_file("questions-qwen.yaml")

/Users/ivanleo/Documents/coding/systematically-improving-rag/cohort_2/.venv/lib/python3.9/site-packages/pydantic_evals/dataset.py:390: UserWarning: Could not determine the generic parameters for <class 'pydantic_evals.dataset.Dataset'>; using `Any` for each. You should explicitly set the generic parameters via `Dataset[MyInputs, MyOutput, MyMetadata]` when serializing or deserializing.
  warnings.warn(


## Conclusion

In this notebook, we covered key metrics like precision and recall, and demonstrated how to generate diverse synthetic questions to benchmark our retrieval system using a Text-2-SQL retrieval system as our example.

In the next notebook, we'll be using these same questions to benchmark different retrieval strategies. This will be followed by Notebook 3 where we'll learn to validate our improvements using bootstrapping and confidence intervals.

Looking ahead to Week 2, we'll leverage these concepts to fine-tune our retrieval models. Using both Cohere's managed re-ranker and the open-source BGE embedding model, we'll see how fine-tuning can improve our recall and MRR metrics on domain-specific queries. 

It's important to note that while synthetic data is valuable for development, it should eventually be augmented with real user queries in production.